In [2]:
!pip install -U datasets fsspec
!pip install transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [3]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
print("Article: ", dataset["train"][0]["article"])
print("Summary: ", dataset["train"][0]["highlights"])  # this is the summary

Article:  LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Deta

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")


def preprocess(example):
    inputs = ["summarize: " + doc for doc in example["article"]]
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["highlights"], max_length=128, truncation=True, padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized = dataset.map(preprocess, batched=True)
small_train = tokenized["train"].shuffle(seed=42).select(range(20000))
small_eval = tokenized["validation"].shuffle(seed=42).select(range(1000))

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./t5_summarization",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    num_train_epochs=4,
    lr_scheduler_type="linear",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_steps=100,
    report_to="none"
)


In [16]:
from transformers import EarlyStoppingCallback

callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    # compute_metrics=compute_metrics,  # using ROUGE
    tokenizer=tokenizer,
    callbacks=callbacks  # optional
)


<ipython-input-18-147e2750142d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.167300,1.120445
2,1.134100,1.112734
3,1.097900,1.110801
4,1.054400,1.110316


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5000, training_loss=1.1337956512451173, metrics={'train_runtime': 3478.9565, 'train_samples_per_second': 22.995, 'train_steps_per_second': 1.437, 'total_flos': 1.082734411776e+16, 'train_loss': 1.1337956512451173, 'epoch': 4.0})

In [23]:
model.save_pretrained("./fine_tuned_t5_summary")

## ROUGE

In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("cnn_dailymail", "3.0.0")
tokenizer = AutoTokenizer.from_pretrained("t5-small")

eval_samples = dataset["validation"].select(range(200))

In [21]:
def preprocess_inputs(example):
    return tokenizer(
        "summarize: " + example["article"],
        return_tensors="pt",
        max_length=512,
        truncation=True,
    )


def decode_summary(token_ids):
    return tokenizer.decode(token_ids, skip_special_tokens=True)

In [24]:
from transformers import AutoModelForSeq2SeqLM

base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_t5_summary")

In [25]:
from tqdm import tqdm


def generate_summaries(model):
    summaries = []
    for example in tqdm(eval_samples):
        inputs = preprocess_inputs(example)
        output = model.generate(**inputs, max_length=128)
        summary = decode_summary(output[0])
        summaries.append(summary)
    return summaries


base_summaries = generate_summaries(base_model)
tuned_summaries = generate_summaries(fine_tuned_model)

references = [ex["highlights"] for ex in eval_samples]

100%|██████████| 200/200 [04:54<00:00,  1.47s/it]


In [29]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=811d1b0a8d2e58807569166a7c39412ed644e7f52e6c3aea9d45a46bdfc5d798
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [30]:
import evaluate

rouge = evaluate.load("rouge")

base_rouge = rouge.compute(
    predictions=base_summaries, references=references, use_stemmer=True
)
tuned_rouge = rouge.compute(
    predictions=tuned_summaries, references=references, use_stemmer=True
)


def print_rouge(scores, name):
    print(f"\n{name} ROUGE Scores:")
    for key in scores:
        print(f"{key}: {scores[key]:.4f}")


print_rouge(base_rouge, "Base T5")
print_rouge(tuned_rouge, "Fine-tuned T5")


Base T5 ROUGE Scores:
rouge1: 0.2993
rouge2: 0.1093
rougeL: 0.2210
rougeLsum: 0.2536

Fine-tuned T5 ROUGE Scores:
rouge1: 0.3306
rouge2: 0.1433
rougeL: 0.2533
rougeLsum: 0.2881
